<a href="https://colab.research.google.com/github/kharlescim/ERT_Project/blob/main/tfdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

currently experimenting with gdb model, this is a quick upload of basic code

going to migrate from basic model to some form of this

In [32]:
# still currently experimenting with gdb model

!pip install tensorflow
!pip install xarray netCDF4
!pip install bottleneck
!pip install sklearn
!pip install -q tensorflow_decision_forests
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import pandas as pd
import keras
from keras.optimizers import SGD
from sklearn.utils import class_weight
from sklearn.metrics import classification_report

import xarray as xr
import numpy as np
import tensorflow_decision_forests as tfdf

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 936, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1069, in get_code
  File "<frozen importlib._bootstrap_external>", line 729, in _compile_bytecode
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 4, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.11/dist-packag

In [16]:
LTD_ds = xr.open_dataset('LTD05.nc')
spei_ds = xr.open_dataset('spei_obs_3D.nc')
obs_ds = xr.open_dataset('obs.nc')

LTD = LTD_ds['LTD']

# Creating new time coordinate
weekly_time = LTD_ds.time.values

# Converting to weekly (method = linear)
spei_weekly = spei_ds.interp(time=weekly_time, method="linear")
obs_weekly = obs_ds.interp(time=weekly_time, method="linear")

# Function to convert raw values to percentiles
# missing values = -999 in obs - might need to alter for proper percentile (7-4)
def to_percentile(ds, dim='time', missing_val = -999.0):

    valid = ds.where(ds != missing_val)
    # Convert each grid point's time series to percentile values.
    return valid.rank(dim=dim, pct=True)

percentiles_spei = to_percentile(spei_weekly)
percentiles_obs = to_percentile(obs_weekly)

# ens = 1, so safe to ignore it from dataset
# testing flattening out entire dataset
spei_df = percentiles_spei.to_dataframe().reset_index()
LTD_df = LTD.to_dataframe().reset_index()
obs_df = (percentiles_obs.to_dataframe().reset_index()).drop(columns=['ens'])
merged_df = pd.merge(spei_df, obs_df, on=['time', 'lat', 'lon'], how='inner')
merged_df = pd.merge(merged_df, LTD_df, on=['time', 'lat', 'lon'], how='inner')

# cleaning up NaN entries
df = merged_df.dropna().copy()

# changing -1 classification to 5 to work with featurespace
df["LTD"] = df["LTD"].replace(-1, 5)

# changing datetime to be usable by TensorFlow, changing LTD from float to int to match example
df["time"] = pd.to_datetime(df["time"]).map(pd.Timestamp.timestamp)

df = df.drop(columns = ["SPI9", "SRI6", "SPEI48", "SRI24", "SMP1", "SMP24", "SPEI6", "SPEI60", "SPEI72", "SRI3", "SPI60", "SPI6", "SMP60", "SRI1", "SPEI3", "SRI60", "SPEI2", "SPI3", "SPEI1", "SMP3", "SPEI36", "SPI24"])

# =====================================================================================================================================================================================
# changing LTD to be type str for tfdf
df["LTD"] = df["LTD"].astype(float).astype(int).astype(str)
val_df = df.sample(frac=0.2, random_state=1337)
train_df = df.drop(val_df.index)

train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="LTD")
val_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(val_df, label="LTD")





Use /tmp/tmpqywaz8jr as temporary training directory
Reading training dataset...
Training dataset read in 0:00:22.395435. Found 3422390 examples.
Training model...
Model trained in 0:12:50.180837
Compiling model...
Model compiled.
None


In [29]:
model = tfdf.keras.RandomForestModel(
    task=tfdf.keras.Task.CLASSIFICATION,
    num_trees=300,
    max_depth=16
)
model.fit(train_dataset)
print(model.make_inspector().label_classes())

Use /tmp/tmphhz4pksr as temporary training directory
Reading training dataset...
Training dataset read in 0:00:22.081342. Found 3422390 examples.
Training model...
Model trained in 0:17:44.289335
Compiling model...
Model compiled.
None


In [30]:

# Classification Report
y_true = val_df["LTD"].astype(float).astype(int).values
X_val_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(val_df.drop(columns=["LTD"]))

y_pred = model.predict(X_val_dataset)

y_pred_labels = np.argmax(y_pred, axis=1)


print(classification_report(y_true, y_pred_labels, digits=3))

856/856 [==============================] - 96s 112ms/step
              precision    recall  f1-score   support

           0      0.840     0.072     0.132     60599
           1      0.641     0.215     0.322     59606
           2      0.526     0.567     0.546     66276
           3      0.605     0.560     0.582     41979
           4      0.766     0.544     0.636     16096
           5      0.856     0.992     0.919    611042

    accuracy                          0.810    855598
   macro avg      0.706     0.492     0.523    855598
weighted avg      0.800     0.810     0.771    855598



In [31]:
print(model.summary())

Model: "random_forest_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (13):
	SMP12
	SMP6
	SMP9
	SPEI12
	SPEI24
	SPEI9
	SPI1
	SPI12
	SRI12
	SRI9
	lat
	lon
	time

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.    "lon"  0.386015 ################
    2. "SPEI12"  0.198298 ######
    3. "SPEI24"  0.155203 ####
    4.    "lat"  0.132561 ###
    5.   "time"  0.130867 ###
    6.  "SPEI9"  0.112655 ##
    7.  "SPI12"  0.098123 #
    8.   "SMP9"  0.096259 #
    9.  "SRI12"  0.089177 
   10.  "SMP12"  0.087992 
   11.   "SMP6"  0.078907 
   12.   "SRI9"  0.075908 
   13.   "SPI1"  0.069832 

Variable Importance: NUM_AS_ROOT:
    1.    "lon" 89.000000 #######